# Stabilized Collective Loan in Marlowe: Working around Protocol Limits


**Executive Summary**

This collective loan includes multiple investors and multiple borrowers, where the pool is supervised by an administrator and where a stabilization fund guarantees returns to investors, subject to a cap on losses. Although this contract is simulatable without difficulty, it would fail on a network like `preview` or `mainnet` due to its exceed the execution-memory limit if it didn't include extra `Notify` terms to split steps into small transactions. Also note that this contract is a proof of principle and that the mathematics underlying it requires review and adjustment. See [stabilized-collective-loan-failure.ipynb](stabilized-collective-loan-failure.ipynb) for an example of how not including these extra `Notify`s causes transactions to fail.


**Highlights**

*   Demonstrates non-blocking decentralized multiplexing of actions by all roles in the contract.
*   Includes a `When` term with 16 `Case` terms, which puts the contract to 3% less than the execution-memory limit.
    *   Involving more than the 1 stabilizer, 2 investors, and 2 borrowers here would require using breaking up this `When` into layers of smaller `When`s.
*   Uses a reference script for the Marlowe validator, in order to keep transaction size under protocol limits.
*   Intercalates `When [Notify` between contract terms, in order to keep transactions under execution memory limits.
*   Execution via the new `marlowe-cli auto-execute` to handle coin selection.

**Roles**

*   Role: Stabilizers
    *   Contributes funds to the stabilizer accounts of the contract.
    *   Stabilizer accounts reimburse Investors for shortfalls when the contract terminates.
        *   Reimbursement capped at the percentage stabilization of the loan pool.
    *   Stabilizers receive net excess loan proceeds when contract terminates.
*   Role: Investors
    *   Contribute liquidity used for lending.
    *   Guaranteed return of principal plus interest, but capped at percentage stabilization in case of severe net underperformance.
    *   May withdraw liquidity at any time.
*   Role: Administrator
    *   Approves loans.
    *   Decides when the contract terminates and settles to all parties.
*   Role: Borrowers
    *   Receives loans.
    *   Repays principal plus interest.

## Pool Operations

![Operations in the stabilized collective loan](stabilized-collective-loan-operations.png)

## Sequence Diagram

![Sequence of interactions in example](stabilized-collective-loan-interaction.svg)

## Preliminaries

### Versions

Make a note of the versions of the Marlowe repository and software.

In [1]:
export TREASURY=treasury

In [2]:
marlowe-cli --version

marlowe-cli 0.0.8.3 @ 3709087ea19ba42b60ffc302373fa28cc9d368dc


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

4dd27deb56d0a140f0df28c9cb6a7f09a9912ec8


### Select the Cardano network

Use the `preview` networks, which is in the Babbage era.

In [5]:
export CARDANO_TESTNET_MAGIC=2
export CARDANO_NODE_SOCKET_PATH=/extra/iohk/networks/preview/node.socket

### Current Time

We'll start the contract at the current time.

In [6]:
NOW=$(($(date -u +%s) * 1000))
echo "$NOW = $(date -u)"

1664078286000 = Sun Sep 25 03:58:06 AM UTC 2022


Compute time constants, for convenience when setting time intervals for transactions.

In [7]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

There is a minimum Ada requirement associated with a native token.

In [8]:
ADA=1000000
MIN_ADA=$((2 * ADA))
echo $MIN_ADA

2000000


### Contract Parameters

Set parameters for the contract. This contract scales to large numbers of lenders and borrowers, but that will involve many transactions and much time if run on-chain.

In [9]:
BASENAME=stabilized-collective-loan

DJED_POLICY=ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
DJED_NAME=DjedUSD

N_STABILIZERS=1
N_INVESTORS=2
N_BORROWERS=2

N_STEPS=20

MINIMUM_STABILIZATION=1000000
MAXIMUM_STABILIZATION=10000000

MINIMUM_INVESTMENT=500000
MAXIMUM_INVESTMENT=5000000

MINIMUM_LOAN=100000
MAXIMUM_LOAN=1000000

PERCENT_STABILIZATION=40
PERCENT_INTEREST=10

TIMEOUT_START=$((NOW + 6 * HOUR))
TIMEOUT_DELTA=$((10 * DAY))
TIMEOUT_FINAL=$((NOW + 1000 * DAY))

ADMINISTRATOR_ROLE=A
STABILIZER_PREFIX=S
INVESTOR_PREFIX=I
BORROWER_PREFIX=B

### Helper functions

#### Role names

In [10]:
function stabilizer() {
  echo "$STABILIZER_PREFIX$1"
}

In [11]:
function investor() {
  echo "$INVESTOR_PREFIX$1"
}

In [12]:
function borrower() {
  echo "$BORROWER_PREFIX$1"
}

#### Datum constituents

In [13]:
function show_contract() {
  jq .tx.contract $BASENAME-$1.marlowe | json2yaml
}

In [14]:
function show_state() {
  jq .tx.state $BASENAME-$1.marlowe | json2yaml
}

In [15]:
function show_choices() {
  jq .tx.state.choices $BASENAME-$1.marlowe | json2yaml
}

In [16]:
function show_nonchoices() {
  jq '.tx.state | {accounts : .accounts, boundValues : .boundValues}' $BASENAME-$1.marlowe | json2yaml
}

#### Query

In [17]:
function query_administrator() {
  cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADMINISTRATOR_ADDR"
}

In [18]:
function query_stabilizer() {
  cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${STABILIZER_ADDR[$1]}"
}

In [19]:
function query_investor() {
  cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${INVESTOR_ADDR[$1]}"
}

In [20]:
function query_borrower() {
  cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${BORROWER_ADDR[$1]}"
}

In [21]:
function query_marlowe() {
  cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$(marlowe-cli contract address)" \
  | sed -n -e "1,2p;/^$1/p"
}

In [22]:
function query_payout() {
  cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$(marlowe-cli role address)" \
  | sed -n -e "1,2p;/^$1/p"
}

#### UTxOs

The following functions simplify UTxO management.

In [23]:
function find_role() {
  marlowe-cli util select --asset-only $ROLES_POLICY.$2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/'
}

In [24]:
function find_ada() {
  marlowe-cli util select --lovelace-only $((10 * ADA)) $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

In [25]:
function find_djed() {
  marlowe-cli util select --asset-only $DJED $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

#### Transactions

In [26]:
function set_validity_interval {
  INVALID_BEFORE=$(jq -r .tx.state.minTime "$BASENAME-$1.marlowe")
  INVALID_HEREAFTER=$((INVALID_BEFORE + 3 * HOUR))
  echo "$INVALID_BEFORE + $((INVALID_HEREAFTER - INVALID_BEFORE)) = $INVALID_HEREAFTER"
}

In [27]:
function execute {

if [[ n -eq 1 ]]
then

TX_ID[$n]=$(
marlowe-cli run auto-execute --marlowe-out-file "$BASENAME-$n.marlowe" \
                             --required-signer "$1" \
                             --change-address "$2" \
                             --out-file "$BASENAME-$n.raw" \
                             --print-stats \
                             --submit=600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)

else

TX_ID[$n]=$(
marlowe-cli run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                             --tx-in-marlowe "${TX_ID[$((n-1))]}#1" \
                             --marlowe-out-file "$BASENAME-$n.marlowe" \
                             --required-signer "$1" \
                             --change-address "$2" \
                             --out-file "$BASENAME-$n.raw" \
                             --print-stats \
                             --submit=600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)

fi

echo
echo "Transaction $n hash: ${TX_ID[$n]}"

}

In [28]:
function withdraw {

marlowe-cli run auto-withdraw --marlowe-file "$BASENAME-$n.marlowe" \
                              --role-name "$1" \
                              --required-signer "$2" \
                              --change-address "$3" \
                              --out-file /dev/null \
                              --submit 600 \
                              --print-stats

}

We have inserted a `When [Notify` before each sub-contract in the contract in order to reduce the execution memory requirements enough so that the contract can be run on-chain. We define an `unpause` function to move beyond these pauses.

In [29]:
function unpause {

for i in `seq 1 $1`
do

echo
echo "----- Unpause $i -----"
echo

n=$((n+1))

set_validity_interval $((n-1))
marlowe-cli run prepare --notify \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats

TX_ID[$n]=$(
marlowe-cli run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                             --tx-in-marlowe "${TX_ID[$((n-1))]}#1" \
                             --marlowe-out-file "$BASENAME-$n.marlowe" \
                             --required-signer "${ADMINISTRATOR_SKEY}" \
                             --change-address "${ADMINISTRATOR_ADDR}" \
                             --out-file "$BASENAME-$n.raw" \
                             --print-stats \
                             --submit=600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)

echo
echo "Transaction $n hash: ${TX_ID[$n]}"

done

}

### Setup the faucet.

In [30]:
FAUCET_SKEY_FILE=$TREASURY/payment.skey
FAUCET_ADDRESS=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDRESS"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Key Derivation

We'll derive all of the keys for stabilizers, investors, and borrowers from a single root private key.

In [31]:
ROOT_PRV=$TREASURY/william-shakespeare.root.prv

For the convenience of viewing transactions in Daedalus, we'll include the same stake credential in all of the addresses.

In [32]:
cardano-wallet key child 1852H/1815H/0H/2/0 < $ROOT_PRV \
| cardano-cli key convert-cardano-address-key --shelley-stake-key --signing-key-file /dev/stdin --out-file /dev/stdout \
| cardano-cli key verification-key --signing-key-file /dev/stdin --verification-key-file /dev/stdout \
| cardano-cli key non-extended-key --extended-verification-key-file /dev/stdin --verification-key-file $TREASURY/william-shakespeare.stake.vkey
cardano-cli stake-address build --testnet-magic $CARDANO_TESTNET_MAGIC --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey

stake_test1urplvp2a7dythh6yxutd0qlzzkncr3gy5ftvxj02d3etafqjugc5h


#### Key for the Administrator

Build the signing keys and payment addresses for the administrator from the `1852H/1815H/0H/0/` sequence of key derivations.

Use a pre-existing key for payment.

In [33]:
ADMINISTRATOR_SKEY=$TREASURY/christopher-marlowe.skey
ADMINISTRATOR_VKEY=$TREASURY/christopher-marlowe.vkey
ADMINISTRATOR_ADDR=$(cat $TREASURY/christopher-marlowe.testnet.address)
echo $ADMINISTRATOR_ADDR

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz


In [34]:
ADMINISTRATOR_SKEY[$j]=tmp/administrator.skey
ADMINISTRATOR_ADDR[$j]=$(
cardano-wallet key child 1852H/1815H/0H/0/0 < $ROOT_PRV \
| cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file $ADMINISTRATOR_SKEY
cardano-cli key verification-key --signing-key-file $ADMINISTRATOR_SKEY --verification-key-file /dev/stdout \
| cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
)
echo "Administrator 1852H/1815H/0H/0/0 = $ADMINISTRATOR_ADDR"

Administrator 1852H/1815H/0H/0/0 = addr_test1qp2l7afky3eqfkrht5f3qgy7x2yek5dejcnpnuqlwywz9twr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqp8jwfw


Send 500 Ada to the administrator.

In [35]:
marlowe-cli util faucet --lovelace "$((500 * ADA))" \
                        --faucet-address "$FAUCET_ADDRESS" \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        --out-file /dev/null \
                        --submit 600 \
                        $ADMINISTRATOR_ADDR

TxId "3a48e2ff92140440e8490ba36a36230f6e2e0044ba92f2a1c79722f4992f2c4a"


#### Keys for Stabilizers

Build the signing keys and payment addresses for the stabilizers from the `1852H/1815H/0H/0/` sequence of key derivations.

In [36]:
mkdir -p tmp
for j in `seq 1 $N_STABILIZERS`
do
  STABILIZER_SKEY[$j]=tmp/stabilizer-$j.skey
  STABILIZER_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$((1 + j)) < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${STABILIZER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${STABILIZER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
  )
  echo "Stabilizer $j = 1852H/1815H/0H/0/$((1 + j)) = ${STABILIZER_ADDR[$j]}"
done

Stabilizer 1 = 1852H/1815H/0H/0/2 = addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp


Send 25 ADA to each stabilizer.

In [37]:
marlowe-cli util faucet --lovelace "$((25 * ADA))" \
                        --faucet-address "$FAUCET_ADDRESS" \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        --out-file /dev/null \
                        --submit 600 \
                        ${STABILIZER_ADDR[@]}

TxId "d15deacc9aabb25dd7eb1398546079685809f343f6e49d86012d528b64d89b1c"


#### Keys for Investors

Build the signing keys and payment addresses for the investors from the `1852H/1815H/0H/0/` sequence of key derivations.

In [38]:
mkdir -p tmp
for j in `seq 1 $N_INVESTORS`
do
  INVESTOR_SKEY[$j]=tmp/investor-$j.skey
  INVESTOR_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$((1 + N_STABILIZERS + j)) < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${INVESTOR_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${INVESTOR_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
  )
  echo "Investor $j = 1852H/1815H/0H/0/$((1 + N_STABILIZERS + j)) = ${INVESTOR_ADDR[$j]}"
done

Investor 1 = 1852H/1815H/0H/0/3 = addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3
Investor 2 = 1852H/1815H/0H/0/4 = addr_test1qqjrjfslev50d5jn6qdeztxsqvuaaefq6e4hu55n59grjj7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqwhulyr


Send 25 ADA to each investor.

In [39]:
marlowe-cli util faucet --lovelace "$((25 * ADA))" \
                        --faucet-address "$FAUCET_ADDRESS" \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        --out-file /dev/null \
                        --submit 600 \
                        ${INVESTOR_ADDR[@]}

TxId "51fbe0451fbcfc57f72946c144aae469bee8c2b92bca53dd60b2917a547fde04"


#### Keys for Borrowers

Build the signing keys and payment addresses for the borrowers from the `1852H/1815H/0H/0/` sequence of key derivations.

In [40]:
mkdir -p tmp
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_SKEY[$j]=tmp/borrower-$j.skey
  BORROWER_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$((1 + N_STABILIZERS + N_INVESTORS + j)) < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${BORROWER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${BORROWER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
  )
  echo "Borrower $j = 1852H/1815H/0H/0/$((1 +N_STABILIZERS + N_INVESTORS + j)) = ${BORROWER_ADDR[$j]}"
done

Borrower 1 = 1852H/1815H/0H/0/5 = addr_test1qqxrzfv0v8l0lahyc5qsgpug66dafp8njl278g0pter7e3xr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqcjuuw0
Borrower 2 = 1852H/1815H/0H/0/6 = addr_test1qqqfzyuqkvrr7ajgk887sqac8lgzatc6el9uxke4x5g5dg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqkmrpwu


Send 25 ADA to each borrower.

In [41]:
marlowe-cli util faucet --lovelace "$((25 * ADA))" \
                        --faucet-address "$FAUCET_ADDRESS" \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        --out-file /dev/null \
                        --submit 600 \
                        ${BORROWER_ADDR[@]}

TxId "de420d4192e780a48cc8742209230e388f5285689d060a271d2bc40a9c5c1b23"


### Mint the Djed

The contract will use fake Djed for payments.

In [42]:
DJED_NAME=DjedUSD

In [43]:
TOTAL_DJED=$((
    N_STABILIZERS * MAXIMUM_STABILIZATION
  + N_INVESTORS   * MAXIMUM_INVESTMENT
  + N_BORROWERS   * MAXIMUM_LOAN
))
echo $TOTAL_DJED

22000000


In [44]:
DJED_POLICY=$(
marlowe-cli util mint --required-signer "$ADMINISTRATOR_SKEY" \
                      --change-address "$ADMINISTRATOR_ADDR" \
                      --expires 1000000011 \
                      --count "$TOTAL_DJED" \
                      --out-file /dev/null \
                      --submit 600 \
                      "$DJED_NAME" \
| sed -e 's/^PolicyID "\(.*\)"$/\1/' \
)
echo 'PolicyID "'$DJED_POLICY'"'

PolicyID "ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee"


In [45]:
DJED=$DJED_POLICY.$DJED_NAME
echo $DJED

ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.DjedUSD


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [46]:
echo $DJED_POLICY.$(echo -n $DJED_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344


### Distribute the Djed.

In [47]:
TXOUT=()
for j in `seq 1 $N_STABILIZERS`
do
  TXOUT+=("--tx-out" "${STABILIZER_ADDR[$j]}+$MIN_ADA+$MAXIMUM_STABILIZATION\ $DJED")
done
for j in `seq 1 $N_INVESTORS`
do
  TXOUT+=("--tx-out" "${INVESTOR_ADDR[$j]}+$MIN_ADA+$MAXIMUM_INVESTMENT\ $DJED")
done
for j in `seq 1 $N_BORROWERS`
do
  TXOUT+=("--tx-out" "${BORROWER_ADDR[$j]}+$MIN_ADA+$MAXIMUM_LOAN\ $DJED")
done

In [48]:
echo marlowe-cli transaction simple --tx-in "$(find_ada  $ADMINISTRATOR_ADDR)" \
                                    --tx-in "$(find_djed $ADMINISTRATOR_ADDR)" \
                                    ${TXOUT[@]} \
                                    --change-address $ADMINISTRATOR_ADDR \
                                    --required-signer $ADMINISTRATOR_SKEY \
                                    --out-file /dev/null \
                                    --submit 600 \
| bash

TxId "ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6"


### Mint the Role Tokens

In [49]:
for j in `seq 1 $N_STABILIZERS`
do
  STABILIZER_ROLE[$j]=$STABILIZER_PREFIX$j
done

In [50]:
for j in `seq 1 $N_INVESTORS`
do
  INVESTOR_ROLE[$j]=$INVESTOR_PREFIX$j
done

In [51]:
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_ROLE[$j]=$BORROWER_PREFIX$j
done

Mint the role tokens.

In [52]:
ROLES_POLICY=$(
marlowe-cli util mint --required-signer "$ADMINISTRATOR_SKEY" \
                      --change-address "$ADMINISTRATOR_ADDR" \
                      --expires 1000000013 \
                      --out-file /dev/null \
                      --submit 600 \
                      $ADMINISTRATOR_ROLE \
                      ${STABILIZER_ROLE[@]} \
                      ${INVESTOR_ROLE[@]} \
                      ${BORROWER_ROLE[@]} \
| sed -e 's/^PolicyID "\(.*\)"$/\1/' \
)
echo 'PolicyID "'$ROLES_POLICY'"'

PolicyID "2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f"


View the role tokens.

In [53]:
query_administrator

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
25943411c5c8bf0a36be667f3269d0fc809b5234f2ea87d642b0e572d56f024d     0        429423109 lovelace + TxOutDatumNone
25943411c5c8bf0a36be667f3269d0fc809b5234f2ea87d642b0e572d56f024d     1        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.41 + TxOutDatumNone
25943411c5c8bf0a36be667f3269d0fc809b5234f2ea87d642b0e572d56f024d     2        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.5331 + TxOutDatumNone
25943411c5c8bf0a36be667f3269d0fc809b5234f2ea87d642b0e572d56f024d     3        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4931 + TxOutDatumNone
25943411c5c8bf0a36be667f3269d0fc809b5234f2ea87d642b0e572d56f024d     4        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4932 + TxOutDatumNone


### Distribute the Role Tokens

In [54]:
TX_IN=($(find_ada  $ADMINISTRATOR_ADDR))
TXOUT=()
for j in `seq 1 $N_STABILIZERS`
do
  TXIN+=("--tx-in" "$(find_role $ADMINISTRATOR_ADDR ${STABILIZER_ROLE[$j]})")
  TXOUT+=("--tx-out" "${STABILIZER_ADDR[$j]}+$MIN_ADA+1\ $ROLES_POLICY.${STABILIZER_ROLE[$j]}")
done
for j in `seq 1 $N_INVESTORS`
do
  TXIN+=("--tx-in" "$(find_role $ADMINISTRATOR_ADDR ${INVESTOR_ROLE[$j]})")
  TXOUT+=("--tx-out" "${INVESTOR_ADDR[$j]}+$MIN_ADA+1\ $ROLES_POLICY.${INVESTOR_ROLE[$j]}")
done
for j in `seq 1 $N_BORROWERS`
do
  TXIN+=("--tx-in" "$(find_role $ADMINISTRATOR_ADDR ${BORROWER_ROLE[$j]})")
  TXOUT+=("--tx-out" "${BORROWER_ADDR[$j]}+$MIN_ADA+1\ $ROLES_POLICY.${BORROWER_ROLE[$j]}")
done

In [55]:
echo marlowe-cli transaction simple ${TXIN[@]} \
                                    ${TXOUT[@]} \
                                    --change-address "$ADMINISTRATOR_ADDR" \
                                    --required-signer "$ADMINISTRATOR_SKEY" \
                                    --out-file /dev/null \
                                    --submit 600 \
| bash

TxId "ef288046881fccdb0a25c608297e00e7684fe428cc888041dbfc5612455dc9b6"


View the role tokens.

In [56]:
echo "UTxOs at the Administor address:"
query_administrator

UTxOs at the Administor address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
25943411c5c8bf0a36be667f3269d0fc809b5234f2ea87d642b0e572d56f024d     0        429423109 lovelace + TxOutDatumNone
25943411c5c8bf0a36be667f3269d0fc809b5234f2ea87d642b0e572d56f024d     1        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.41 + TxOutDatumNone
ef288046881fccdb0a25c608297e00e7684fe428cc888041dbfc5612455dc9b6     0        39784823 lovelace + TxOutDatumNone


In [57]:
for j in `seq 1 $N_STABILIZERS`
do
  echo "UTxOs at the Stabilizer $j address:"
  query_stabilizer $j
  echo
done

UTxOs at the Stabilizer 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     1        2000000 lovelace + 10000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
d15deacc9aabb25dd7eb1398546079685809f343f6e49d86012d528b64d89b1c     1        25000000 lovelace + TxOutDatumNone
ef288046881fccdb0a25c608297e00e7684fe428cc888041dbfc5612455dc9b6     1        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.5331 + TxOutDatumNone



In [58]:
for j in `seq 1 $N_INVESTORS`
do
  echo "UTxOs at the Investor $j address:"
  query_investor $j
  echo
done

UTxOs at the Investor 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
51fbe0451fbcfc57f72946c144aae469bee8c2b92bca53dd60b2917a547fde04     1        25000000 lovelace + TxOutDatumNone
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     2        2000000 lovelace + 5000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
ef288046881fccdb0a25c608297e00e7684fe428cc888041dbfc5612455dc9b6     2        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4931 + TxOutDatumNone

UTxOs at the Investor 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
51fbe0451fbcfc57f72946c144aae469bee8c2b92bca53dd60b2917a547fde04     2        25000000 lovelace + TxOutD

In [59]:
for j in `seq 1 $N_BORROWERS`
do
  echo "UTxOs at the Borrower $j address:"
  query_borrower $j
  echo
done

UTxOs at the Borrower 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     4        2000000 lovelace + 1000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
de420d4192e780a48cc8742209230e388f5285689d060a271d2bc40a9c5c1b23     1        25000000 lovelace + TxOutDatumNone
ef288046881fccdb0a25c608297e00e7684fe428cc888041dbfc5612455dc9b6     4        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4231 + TxOutDatumNone

UTxOs at the Borrower 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     5        2000000 lovelace + 1000000

### Publish the Marlowe reference script

In [60]:
marlowe-cli transaction publish --required-signer "$FAUCET_SKEY_FILE" \
                                --change-address "$FAUCET_ADDRESS" \
                                --permanently-without-staking \
                                --out-file /dev/null \
                                --submit 600


Fee: Lovelace 844069
Size: 15409 / 16384 = 94%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%

Marlowe script published at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Marlowe script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Marlowe ref script UTxO min ADA: Lovelace 12434350

Payout script published at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Payout script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Payout ref script UTxO min ADA: Lovelace 12434350


In [61]:
PUBLISHED=$(marlowe-cli transaction find-published)


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"


In [62]:
json2yaml <(echo "$PUBLISHED")

marlowe:
  hash: 6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0
  txIn: 087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1
payout:
  hash: 49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3
  txIn: 087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2


## Generate the Contract

Run a Haskell program that generates the contract.

In [63]:
runhaskell << EOI

{-# LANGUAGE BlockArguments     #-}
{-# LANGUAGE NumericUnderscores #-}
{-# LANGUAGE OverloadedStrings  #-}
{-# LANGUAGE RecordWildCards    #-}


module Main (
  main
) where


import Control.Monad ((<=<))
import Control.Monad.Writer (Writer, runWriter, tell)
import Data.Aeson ((.=), encodeFile,object)
import Data.Default (Default(..))
import Data.Foldable (foldrM)
import Data.Maybe (fromMaybe)
import Data.String (fromString)
import Language.Marlowe.Core.V1.Semantics.Types
import Plutus.V1.Ledger.Api (BuiltinByteString, POSIXTime(..), TokenName)
import Language.Marlowe.CLI.Merkle (deepMerkleize)
import Language.Marlowe.CLI.Types (Continuations)


data Scenario =
  Scenario
  {
    nSteps               :: Int
  , timeoutStart         :: Integer
  , timeoutDelta         :: Integer
  , timeoutFinal         :: POSIXTime
  , nStabilizers         :: Int
  , nInvestors           :: Int
  , nBorrowers           :: Int
  , administratorName    :: String
  , stabilizerPrefix     :: String
  , investorPrefix       :: String
  , borrowerPrefix       :: String
  , stabilizeBound       :: [Bound]
  , investBound          :: [Bound]
  , loanBound            :: [Bound]
  , percentInterest      :: Integer
  , percentStabilization :: Integer
  , djed                 :: Token
  }

instance Default Scenario where
  def = 
    Scenario
    {
      nSteps               = $N_STEPS
    , timeoutStart         = $TIMEOUT_START
    , timeoutDelta         = $TIMEOUT_DELTA
    , timeoutFinal         = POSIXTime $TIMEOUT_FINAL
    , nStabilizers         = $N_STABILIZERS
    , nInvestors           = $N_INVESTORS
    , nBorrowers           = $N_BORROWERS
    , administratorName    = "$ADMINISTRATOR_ROLE"
    , stabilizerPrefix     = "$STABILIZER_PREFIX"
    , investorPrefix       = "$INVESTOR_PREFIX"
    , borrowerPrefix       = "$BORROWER_PREFIX"
    , stabilizeBound       = [Bound $MINIMUM_STABILIZATION $MAXIMUM_STABILIZATION]
    , investBound          = [Bound $MINIMUM_INVESTMENT    $MAXIMUM_INVESTMENT   ]
    , loanBound            = [Bound $MINIMUM_LOAN          $MAXIMUM_LOAN         ]
    , percentInterest      = $PERCENT_INTEREST
    , percentStabilization = $PERCENT_STABILIZATION
    , djed                 = Token "$DJED_POLICY" "$DJED_NAME"
    }


main :: IO ()
main =
  let
    (contract, continuations) = buildContract def
  in
    encodeFile "$BASENAME.merkleization"
      $ object
        [
          "contract"      .= contract
        , "continuations" .= continuations
        ]


buildContract :: Scenario -> (Contract, Continuations)
buildContract scenario@Scenario{..} =
  let
    timeouts = [POSIXTime $ timeoutStart + toInteger i * timeoutDelta | i <- [1..nSteps]]
  in
    runWriter $ foldrM (makeStep scenario) Close timeouts


defaultBound :: [Bound]
defaultBound = [Bound 1 1_000_000_000_000]


makeParty :: String -> Int -> Party
makeParty name i = Role . fromString $ name <> show i


deposit :: Scenario
        -> String
        -> String
        -> Int
        -> Maybe Party
        -> [Bound]
        -> (Value Observation -> Contract -> Writer Continuations Contract)
        -> Contract
        -> Writer Continuations [Case Contract]
deposit Scenario{..} name prefix i recipient bound f continuation =
  let
    party = makeParty prefix i
    party' = fromMaybe party recipient
    choice = ChoiceId (fromString name) party
    value = ChoiceValue choice
  in
    sequence
      [
        pure
          $ Case (Choice choice bound)
            continuation
      , Case (Deposit party' party djed value)
          <$> f value continuation
      ]


stabilization :: ValueId
stabilization = ValueId "s"


stabilize :: Scenario -> Int -> Contract -> Writer Continuations [Case Contract]
stabilize scenario@Scenario{..} i =
  deposit scenario "S" stabilizerPrefix i Nothing stabilizeBound
    (stabilization =+)
 

assets :: ValueId
assets = ValueId "a"


equity :: Int -> ValueId
equity = ValueId . ("e" <>) . fromString . show


invest :: Scenario -> Int -> Contract -> Writer Continuations [Case Contract]
invest scenario@Scenario{..} i =
  deposit scenario "I" investorPrefix i Nothing investBound
    $ \value -> (assets   =+ value)
                  <=< (equity i =+ value)


withdraw :: Scenario
         -> Int
         -> Contract
         -> Writer Continuations [Case Contract]
withdraw Scenario{..} i continuation =
  let
    party = makeParty investorPrefix i
    choice = ChoiceId "W" party
    value = ChoiceValue choice
    funds = AvailableMoney party djed
  in
    (: [])
      . Case (Choice choice defaultBound)
          <$> do
                c <-
                  mPay party (Party party) djed value
                    =<< assets   =- value
                    =<< (equity i =- value)
                        continuation
                c' <-
                  mPay party (Party party) djed funds
                    =<< assets   =- funds
                    =<< (equity i =- funds)
                        continuation
                mIf (value @<= funds) c c'


lend :: Scenario
     -> Party
     -> Value Observation
     -> Value Observation
     -> Int
     -> Contract
     -> Writer Continuations Contract
lend Scenario{..} borrower principal interest i =
  let
    party = makeParty investorPrefix i
    funds = AvailableMoney party djed
    assets' = UseValue assets
    principal' = (principal @* funds) @/ assets'
    interest'  = (interest  @* funds) @/ assets'
  in
    mPay party (Party borrower) djed principal'
      <=< equity i =+ interest'


liabilities :: ValueId
liabilities = ValueId "l"


loan :: Scenario
     -> Int
     -> Contract
     -> Writer Continuations [Case Contract]
loan scenario@Scenario{..} i continuation =
  let
    party = Role $ fromString administratorName
    borrower = makeParty borrowerPrefix i
    choice = ChoiceId (fromString $ "L" <> show i) party
    principal = ChoiceValue choice
    assets' = UseValue assets
    stabilization' = UseValue stabilization
  in
    (: [])
      . Case (Choice choice loanBound)
      <$> let
            interest = principal @% percentInterest
            debt = principal @+ interest
          in
            do
              c <-
                flip (foldrM $ lend scenario borrower principal interest) [1..nInvestors]
                  =<< liabilities =+ debt
                  =<< (assets      =- principal)
                      continuation
              mIf ((principal @<= assets') @&& (stabilization' @>= (debt @% percentStabilization)))
                c continuation


restore :: Scenario
        -> Value Observation
        -> Int
        -> Contract
        -> Writer Continuations Contract
restore Scenario{..} repayment i =
  let
    administrator = Role $ fromString administratorName
    party = makeParty investorPrefix i
    equity' = UseValue $ equity i
    assets' = AvailableMoney party djed
    liabilities' = UseValue liabilities
    share = (repayment @* (equity' @- assets')) @/ liabilities'
  in
    mPay administrator (Account party) djed share


repay :: Scenario
      -> Int
      -> Contract
      -> Writer Continuations [Case Contract]
repay scenario@Scenario{..} i =
  deposit scenario "R" borrowerPrefix i (Just . Role $ fromString administratorName) defaultBound
    $ \repayment -> flip (foldrM $ restore scenario repayment) [1..nInvestors]
                      <=< (liabilities =- repayment)
                      <=< (assets      =+ repayment)


gather :: Scenario
       -> Value Observation
       -> ValueId
       -> String
       -> Int
       -> Contract
       -> Writer Continuations Contract
gather Scenario{..} amount basis prefix i = 
  let
    administrator = Role $ fromString administratorName
    party = makeParty prefix i
    funds = AvailableMoney party djed
    basis' = UseValue basis
    share = (amount @* funds) @/ basis'
  in
    mPay party (Account administrator) djed share


scatter :: Scenario
        -> ValueId
        -> String
        -> Int
        -> Contract
        -> Writer Continuations Contract
scatter Scenario{..} basis prefix i =
  let
    administrator = Role $ fromString administratorName
    party = makeParty prefix i
    funds = AvailableMoney party djed
    basis' = UseValue basis
    amount = AvailableMoney administrator djed
    share = (amount @* funds) @/ basis'
  in
    mPay administrator (Account party) djed share


terminate :: Scenario
          -> Writer Continuations [Case Contract]
terminate scenario@Scenario{..} =
  let
    party = Role $ fromString administratorName
    choice = ChoiceId "T" party
    liabilities' = UseValue liabilities
  in
    (: [])
      . Case (Choice choice [Bound 1 1])
        <$> let
              payout party = mPay party (Party party) djed (AvailableMoney party djed)
              exit =
                flip (foldrM $ payout . makeParty stabilizerPrefix) [1..nStabilizers]
                  =<< flip (foldrM $ payout . makeParty investorPrefix) [1..nInvestors]
                  =<< payout (Role $ fromString administratorName)
                      Close
            in
              do
                c <-
                    flip (foldrM $ gather  scenario liabilities' stabilization stabilizerPrefix) [1..nStabilizers]
                      =<< flip (foldrM $ scatter scenario assets investorPrefix  ) [1..nInvestors  ]
                      =<< exit
                c' <-
                    flip (foldrM $ gather  scenario (NegValue liabilities') assets investorPrefix  ) [1..nInvestors  ]
                      =<< flip (foldrM $ scatter scenario stabilization stabilizerPrefix) [1..nStabilizers]
                      =<< exit
                mIf (liabilities' @>= Constant 0) c c'


makeStep :: Scenario
         -> Timeout
         -> Contract
         -> Writer Continuations Contract
makeStep scenario@Scenario{..} timeout continuation =
  do
    cs <-
        fmap concat . sequence
          $  [stabilize scenario i continuation | i <- [1..nStabilizers]]
          <> [invest    scenario i continuation | i <- [1..nInvestors]  ]
          <> [withdraw  scenario i continuation | i <- [1..nInvestors]  ]
          <> [loan      scenario i continuation | i <- [1..nBorrowers]  ]
          <> [repay     scenario i continuation | i <- [1..nBorrowers]  ]
          <> [terminate scenario                                        ]
    c <- merkleizeTimeout timeoutFinal continuation
    deepMerkleize
      $ When cs timeout c


merkleizeTimeout :: Timeout
                 -> Contract
                 -> Writer Continuations Contract
merkleizeTimeout timeout continuation =
  deepMerkleize
    $ When
      [
        Case (Notify TrueObs )
        continuation
      ]
      timeout
      Close


pause :: Contract
      -> Writer Continuations Contract
pause = merkleizeTimeout $ POSIXTime $TIMEOUT_FINAL


mPay :: AccountId
     -> Payee
     -> Token
     -> Value Observation
     -> Contract
     -> Writer Continuations Contract
mPay a p t v c = pause $ Pay a p t v c


mIf :: Observation
    -> Contract
    -> Contract
    -> Writer Continuations Contract
mIf o c c' = pause $ If o c c'


mWhen :: [Case Contract]
      -> Timeout
      -> Contract
      -> Writer Continuations Contract
mWhen cs t c = pause $ When cs t c


mLet :: ValueId
     -> Value Observation
     -> Contract
     -> Writer Continuations Contract
mLet i v c = pause $ Let i v c



(=+) :: ValueId -> Value Observation -> (Contract -> Writer Continuations Contract)
variable =+ value = mLet variable $ UseValue variable @+ value

(=-) :: ValueId -> Value Observation -> (Contract -> Writer Continuations Contract)
variable =- value = mLet variable $ UseValue variable @- value

(=:) :: ValueId -> Integer -> (Contract -> Writer Continuations Contract)
variable =: value = mLet variable $ Constant value

(@%) :: Value Observation -> Integer -> Value Observation
value @% rate = DivValue (MulValue value $ Constant rate) (Constant 100)

(@+%) :: Value Observation -> Integer -> Value Observation
value @+% rate = DivValue (MulValue value (AddValue (Constant rate) $ Constant 0)) (Constant 100)

(@+) :: Value Observation -> Value Observation -> Value Observation
x @+ y =AddValue x y 

(@-) :: Value Observation -> Value Observation -> Value Observation
x @- y = SubValue x y

(@*) :: Value Observation -> Value Observation -> Value Observation
x @* y = MulValue x y

(@/) :: Value Observation -> Value Observation -> Value Observation
x @/ y = DivValue x y

(@<) :: Value Observation -> Value Observation -> Observation
x @< y = ValueLT x y

(@<=) :: Value Observation -> Value Observation -> Observation
x @<= y = ValueLE x y

(@>) :: Value Observation -> Value Observation -> Observation
x @> y = ValueGT x y

(@>=) :: Value Observation -> Value Observation -> Observation
x @>= y = ValueGE x y

(@==) :: Value Observation -> Value Observation -> Observation
x @== y = ValueEQ x y

(@&&) :: Observation -> Observation -> Observation
x @&& y = AndObs x y

(@||) :: Observation -> Observation -> Observation
x @|| y = OrObs x y

EOI

How large is the contract?

In [64]:
ls -lh $BASENAME.merkleization

-rw-rw-r-- 1 bbush bbush-upg 1.5M Sep 24 22:02 stabilized-collective-loan.merkleization


How many `Case` statements does it contain?

In [65]:
sed -e 's/case/&\n/g' $BASENAME.merkleization | grep case | wc -l

5104


How many merkleizations does it contain?

In [66]:
jq '.continuations | length' $BASENAME.merkleization

1031


Separate the contract from the merkleizations.

In [67]:
jq .contract $BASENAME.merkleization > $BASENAME-1.contract

## Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [68]:
yaml2json << EOI > $BASENAME-1.state
accounts:
- - - role_token: $ADMINISTRATOR_ROLE
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [69]:
cat $BASENAME-1.state

{"accounts":[[[{"role_token":"A"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1664078286000}


## Simulate and Execute the Contract

### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract. Note that we merkleization the contract because of its large size.

In [70]:
n=1

In [71]:
marlowe-cli run initialize --roles-currency "$ROLES_POLICY" \
                           --contract-file "$BASENAME-$n.contract" \
                           --state-file    "$BASENAME-$n.state" \
                           --out-file      "$BASENAME-$n.partial" \
                           --print-stats


Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


Manually merge the continuations into the `.marlowe` file.

In [72]:
jq -s '.[0] * {tx : .[1]}' $BASENAME-1.partial $BASENAME.merkleization > $BASENAME-1.marlowe

The minimum Ada deposit has been recorded.

In [73]:
show_state $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1664078286000


Submit the transaction.

In [74]:
execute "$ADMINISTRATOR_SKEY" "$ADMINISTRATOR_ADDR"


Fee: Lovelace 248177
Size: 1735 / 16384 = 10%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%

Transaction 1 hash: 24f2dddbd9bd89c8af94e27b71d37708b4f92fe103200c516156e1a3cc391d21


The funds are at the Marlowe script address.

In [75]:
query_marlowe "${TX_ID[$n]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
24f2dddbd9bd89c8af94e27b71d37708b4f92fe103200c516156e1a3cc391d21     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "716ee75afc7960b27ec6042528536e6d3f3edbb45f5070420d8005384dbbe495"


### 2. Stabilzer 1 deposits 5M Djed

#### Choose the deposit

In [76]:
STABILIZER_DEPOSIT=5000000

In [77]:
n=$((n+1))

In [78]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [79]:
marlowe-cli run prepare --choice-party "$(stabilizer 1)" \
                        --choice-name S \
                        --choice-number "$STABILIZER_DEPOSIT" \
                        --invalid-before "$INVALID_BEFORE"  \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 1453


In [80]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000


Submit the transaction.

In [81]:
execute "${STABILIZER_SKEY[1]}" "${STABILIZER_ADDR[1]}"


Fee: Lovelace 1180623
Size: 4950 / 16384 = 30%
Execution units:
  Memory: 10013860 / 14000000 = 71%
  Steps: 2893947408 / 10000000000 = 28%

Transaction 2 hash: 5a084691ac2f084ad50b16f025147a457bf16db2bb42474a0a8845ac500b3a17


#### Make the deposit

In [82]:
n=$((n+1))

In [83]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [84]:
marlowe-cli run prepare --deposit-account "$(stabilizer 1)" \
                        --deposit-party "$(stabilizer 1)" \
                        --deposit-token "$DJED" \
                        --deposit-amount "$STABILIZER_DEPOSIT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 217


In [85]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
boundValues: []


Submit the transaction.

In [86]:
execute "${STABILIZER_SKEY[1]}" "${STABILIZER_ADDR[1]}"


Fee: Lovelace 982397
Size: 2676 / 16384 = 16%
Execution units:
  Memory: 8885286 / 14000000 = 63%
  Steps: 2373903516 / 10000000000 = 23%

Transaction 3 hash: 1acc62e2c56a56c353a509e9b2a83801ebd0a18c8640ec11f5e140f1117580b9


The funds have been added to the script address.

In [87]:
query_marlowe ${TX_ID[$n]}

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
1acc62e2c56a56c353a509e9b2a83801ebd0a18c8640ec11f5e140f1117580b9     1        2000000 lovelace + 5000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "aff66450f84a287edd17948ffe851fe651d1832b32bb8ba4d3f3643ec16daebe"


### 4. Unpause the contract using `INotify`

We have inserted a `When [Notify` before each sub-contract in the contract in order to reduce the execution memory requirements enough so that the contract can be run on-chain. We define an `unpause` function to move beyond these pauses.

Now advance the contract.

In [88]:
unpause 1


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 1522

Fee: Lovelace 1007410
Size: 3731 / 16384 = 22%
Execution units:
  Memory: 8548972 / 14000000 = 61%
  Steps: 2469419701 / 10000000000 = 24%

Transaction 4 hash: f335fbfade55b77ac4ca41ec42c5667a047307839d317f68168d93800c3d85f7


### 5. Investor 1 deposits 1M Djed

#### Choose the deposit

In [89]:
INVESTOR_DEPOSIT=1000000

In [90]:
n=$((n+1))

In [91]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [92]:
marlowe-cli run prepare --choice-party "$(investor 1)" \
                        --choice-name I \
                        --choice-number "$INVESTOR_DEPOSIT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 1540


In [93]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000


Submit the transaction.

In [94]:
execute "${INVESTOR_SKEY[1]}" "${INVESTOR_ADDR[1]}"


Fee: Lovelace 1288139
Size: 5170 / 16384 = 31%
Execution units:
  Memory: 11308804 / 14000000 = 80%
  Steps: 3214577154 / 10000000000 = 32%

Transaction 5 hash: 32b7981c1372d17b01a82e440c1658694c8449b40dc35ffd03573106b97ffeb1


#### Make the deposit

In [95]:
n=$((n+1))

In [96]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [97]:
marlowe-cli run prepare --deposit-account "$(investor 1)" \
                        --deposit-party "$(investor 1)" \
                        --deposit-token "$DJED" \
                        --deposit-amount "$INVESTOR_DEPOSIT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 304


In [98]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 1000000
boundValues:
- - s
  - 5000000


Submit the transaction.

In [99]:
execute "${INVESTOR_SKEY[1]}" "${INVESTOR_ADDR[1]}"


Fee: Lovelace 1062822
Size: 2850 / 16384 = 17%
Execution units:
  Memory: 9844778 / 14000000 = 70%
  Steps: 2615329139 / 10000000000 = 26%

Transaction 6 hash: a6c605ec81e8b3c5d77598683b6c01b49dcc484ca2e9b7bf6a8cbf30ca49ddd1


The funds have been added to the script address.

In [100]:
query_marlowe ${TX_ID[$n]}

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a6c605ec81e8b3c5d77598683b6c01b49dcc484ca2e9b7bf6a8cbf30ca49ddd1     1        2000000 lovelace + 6000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "a668d1395aa2ed6ed6a61fdcc4b008f22ecbc36f9581676ae14ffb101414ac7e"


### 6. Unpause

In [101]:
unpause 2


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 315

Fee: Lovelace 671877
Size: 1317 / 16384 = 8%
Execution units:
  Memory: 5744892 / 14000000 = 41%
  Steps: 1532917629 / 10000000000 = 15%

Transaction 7 hash: 9b439de448f6a6f9e5621dbbfafbb31906adfd3cef1ab7a57fccde4031692a97

----- Unpause 2 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 1621

Fee: Lovelace 1066668
Size: 3930 / 16384 = 23%
Execution units:
  Memory: 9208362 / 14000000 = 65%
  Steps: 2642164185 / 10000000000 = 26%

Transaction 8 hash: 175cb500ff4c6523cabf90b60a9377d2e02b98f65df14b87102738d56f2621cf


### 7. Investor 2 deposits 500k Djed

#### Choose the deposit

In [102]:
INVESTOR_DEPOSIT=500000

In [103]:
n=$((n+1))

In [104]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [105]:
marlowe-cli run prepare --choice-party "$(investor 2)" \
                        --choice-name I \
                        --choice-number "$INVESTOR_DEPOSIT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 1639


In [106]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: I
    choice_owner:
      role_token: I2
  - 500000


Submit the transaction.

In [107]:
execute "${INVESTOR_SKEY[2]}" "${INVESTOR_ADDR[2]}"


Fee: Lovelace 1336012
Size: 5368 / 16384 = 32%
Execution units:
  Memory: 11820856 / 14000000 = 84%
  Steps: 3347949410 / 10000000000 = 33%

Transaction 9 hash: f3f39042a88d871c9a16f72bbdd67448177c049fdd9868e54fa702eaaa12131a


#### Make the deposit

In [108]:
n=$((n+1))

In [109]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [110]:
marlowe-cli run prepare --deposit-account "$(investor 2)" \
                        --deposit-party "$(investor 2)" \
                        --deposit-token "$DJED" \
                        --deposit-amount "$INVESTOR_DEPOSIT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 403


In [111]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 1000000
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 500000
boundValues:
- - s
  - 5000000
- - a
  - 1000000
- - e1
  - 1000000


Submit the transaction.

In [112]:
execute "${INVESTOR_SKEY[2]}" "${INVESTOR_ADDR[2]}"


Fee: Lovelace 1115136
Size: 3048 / 16384 = 18%
Execution units:
  Memory: 10417524 / 14000000 = 74%
  Steps: 2761706543 / 10000000000 = 27%

Transaction 10 hash: a596cc51e73258a84796ba3fcd9f49e43e3da2f8e916726ca8fa82bb34ab996a


The funds have been added to the script address.

In [113]:
query_marlowe ${TX_ID[$n]}

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a596cc51e73258a84796ba3fcd9f49e43e3da2f8e916726ca8fa82bb34ab996a     1        2000000 lovelace + 6500000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "7843d1e4761aa503b1dc1f06d28f78ed57b7201113b3d06a521ed0c283d4114c"


### 8. Unpause

In [114]:
unpause 2


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Warnings:
  TransactionShadowing "a" 1000000 1500000
Datum size: 403

Fee: Lovelace 729638
Size: 1504 / 16384 = 9%
Execution units:
  Memory: 6394836 / 14000000 = 45%
  Steps: 1699780206 / 10000000000 = 16%

Transaction 11 hash: 31994c5cf2d4aa6e86c4de042c3d24bc6a7185b9ea115bcb387e4a41dcce7b99

----- Unpause 2 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 1709

Fee: Lovelace 1113650
Size: 4106 / 16384 = 25%
Execution units:
  Memory: 9723668 / 14000000 = 69%
  Steps: 2773993087 / 10000000000 = 27%

Transaction 12 hash: 082731996a14fa792e98851853af6085b6644e239946844702b44c4adbddba53


### 9. Administrator authorizes loan of 900k Djed to Borrower 1

In [115]:
LOAN_AMOUNT=900000

In [116]:
n=$((n+1))

In [117]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [118]:
marlowe-cli run prepare --choice-party "$ADMINISTRATOR_ROLE" \
                        --choice-name L1 \
                        --choice-number "$LOAN_AMOUNT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 433


In [119]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 1000000
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 500000
boundValues:
- - s
  - 5000000
- - a
  - 1500000
- - e1
  - 1000000
- - e2
  - 500000


Submit the transaction.

In [120]:
execute "$ADMINISTRATOR_SKEY" "$ADMINISTRATOR_ADDR"


Fee: Lovelace 935359
Size: 2919 / 16384 = 17%
Execution units:
  Memory: 8250234 / 14000000 = 58%
  Steps: 2204706729 / 10000000000 = 22%

Transaction 13 hash: 2866696e6e94d7ed5fa73b68ed43f72247b728889c3ef5ae04a39da70be45bbd


### 10. Unpause

In [121]:
unpause 7


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 433

Fee: Lovelace 1143340
Size: 3015 / 16384 = 18%
Execution units:
  Memory: 10648952 / 14000000 = 76%
  Steps: 3111099962 / 10000000000 = 31%

Transaction 14 hash: f767f627b4e16387eda2dc996b5790af32ec5eef2b7cdbbab5fea1e4e5421df8

----- Unpause 2 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 433
Payment 1
  Acccount: "I1"
  Payee: Party "B1"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 600000

Fee: Lovelace 1012449
Size: 1834 / 16384 = 11%
Execution units:
  Memory: 9892532 / 14000000 = 70%
  Steps: 2621746350 / 10000000000 = 26%

Transaction 15 hash: 02d364dace7be1afb9e8ac05843d957453d4c31e12af10c36ac0617e0224c244

----- Unpause 3 -----

1664078286000 + 10800000 = 1664089086000

Warnings:
  TransactionShadowing "e1" 1000000 1024000
Datum size: 433

Fee: Lovelace 835430
Size: 1657 / 16384 = 10%
Execution units:
  Memory: 7627716 / 140

The funds have been paid from the Marlowe script address.

In [122]:
query_marlowe "${TX_ID[$n]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2a273e7f71e6be6c0cae9cdbf5834547c7ae29488fc66b34f9ad623f524f1afa     1        2000000 lovelace + 5600000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "32990c5ebf9685d34c62a0722c5a0106aef618bf30cf60542df4e7b3db747601"


And paid to the payout script address in two installments (one from each investor's account).

In [123]:
query_payout "${TX_ID[$((n-5))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
02d364dace7be1afb9e8ac05843d957453d4c31e12af10c36ac0617e0224c244     2        1215420 lovelace + 600000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "f22bf7dae0dae200d80484966a3e4e1074323d87e55011ec025b1fd6a66a5ae6"


In [124]:
query_payout "${TX_ID[$((n-3))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
abdda9b830b8168cda7c3d66e9e57da404685623bdbc37ba522ab8c1b92ea594     2        1215420 lovelace + 300000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "f22bf7dae0dae200d80484966a3e4e1074323d87e55011ec025b1fd6a66a5ae6"


### 11. Borrower 1 withdraws their 900k Djed

Before withdrawing the loan, Borrower 1 has 1M Djed.

In [125]:
query_borrower 1

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     4        2000000 lovelace + 1000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
de420d4192e780a48cc8742209230e388f5285689d060a271d2bc40a9c5c1b23     1        25000000 lovelace + TxOutDatumNone
ef288046881fccdb0a25c608297e00e7684fe428cc888041dbfc5612455dc9b6     4        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4231 + TxOutDatumNone


Submit the transaction to withdraw the loan from the payout validator's address.

In [126]:
withdraw "$(borrower 1)" "${BORROWER_SKEY[1]}" "${BORROWER_ADDR[1]}"


Fee: Lovelace 623766
Size: 603 / 16384 = 3%
Execution units:
  Memory: 5437140 / 14000000 = 38%
  Steps: 1486014138 / 10000000000 = 14%
TxId "147fd7eeb9e3ef75a5596f0da7724b86070aba385fb4543e40464407996d7de4"


After withdrawing the loan, Borrower 1 has an additional 900k Djed.

In [127]:
query_borrower 1

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
147fd7eeb9e3ef75a5596f0da7724b86070aba385fb4543e40464407996d7de4     0        26376234 lovelace + TxOutDatumNone
147fd7eeb9e3ef75a5596f0da7724b86070aba385fb4543e40464407996d7de4     1        2430840 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4231 + 900000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     4        2000000 lovelace + 1000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone


The funds are no longer present at the payout script address.

In [128]:
query_payout "${TX_ID[$((n-6))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [129]:
query_payout "${TX_ID[$((n-4))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


### 12. Administrator authorizes loan of 600k Djed to Borrower 2

In [130]:
LOAN_AMOUNT=600000

In [131]:
n=$((n+1))

In [132]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [133]:
marlowe-cli run prepare --choice-party "$ADMINISTRATOR_ROLE"\
                        --choice-name L2 \
                        --choice-number "$LOAN_AMOUNT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 462


In [134]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 400000
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 200000
boundValues:
- - s
  - 5000000
- - a
  - 600000
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 990000


Submit the transaction.

In [135]:
execute "$ADMINISTRATOR_SKEY" "$ADMINISTRATOR_ADDR"


Fee: Lovelace 1015004
Size: 3013 / 16384 = 18%
Execution units:
  Memory: 9181856 / 14000000 = 65%
  Steps: 2444796561 / 10000000000 = 24%

Transaction 21 hash: bf3ff2db4345d8702fe9cd4ee7d95da424f366c15c87480fa6ae2138f63eae0b


### 13. Unpause

In [136]:
unpause 7


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 462

Fee: Lovelace 1152461
Size: 3073 / 16384 = 18%
Execution units:
  Memory: 10730136 / 14000000 = 76%
  Steps: 3137241389 / 10000000000 = 31%

Transaction 22 hash: ddc53a0dead66ef64c46b32285937d47822ce97997d69c7631814ff062c7ab04

----- Unpause 2 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 404
Payment 1
  Acccount: "I1"
  Payee: Party "B2"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 400000

Fee: Lovelace 975741
Size: 1834 / 16384 = 11%
Execution units:
  Memory: 9400721 / 14000000 = 67%
  Steps: 2506206178 / 10000000000 = 25%

Transaction 23 hash: 858869352fcfc16d4906ca9b2506f0c4b0148de4338643bc24f06471b0ea610d

----- Unpause 3 -----

1664078286000 + 10800000 = 1664089086000

Warnings:
  TransactionShadowing "e1" 1024000 1024000
Datum size: 404

Fee: Lovelace 810574
Size: 1599 / 16384 = 9%
Execution units:
  Memory: 7330054 / 14000

The funds have been paid from the Marlowe script address.

In [137]:
query_marlowe "${TX_ID[$n]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3af1617b87e13d0c1db4c9c64d40b76f77537c7cc8f439dc75942d57e452404c     1        2000000 lovelace + 5000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "15166d5b4ced31103e4ca92c2538ca5e2ae48053bb47a0fcf7dded41bcf8adc1"


And paid to the payout script address in two installments (one from each investor's account).

In [138]:
query_payout "${TX_ID[$((n-5))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
858869352fcfc16d4906ca9b2506f0c4b0148de4338643bc24f06471b0ea610d     2        1215420 lovelace + 400000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "6816962b8dbfd16219f3c1d294ad0f7e1212995ba2d0c7e947c33b583c1151d8"


In [139]:
query_payout "${TX_ID[$((n-3))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
267870a0b5521711cbf54e88e90a04eb9230a2c468015be3269d5b8835a71ae6     2        1215420 lovelace + 200000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "6816962b8dbfd16219f3c1d294ad0f7e1212995ba2d0c7e947c33b583c1151d8"


### 14. Borrower 2 withdraws their 600k Djed

Before withdrawing the loan, Borrower 2 has 1M Djed.

In [140]:
query_borrower 2

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     5        2000000 lovelace + 1000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
de420d4192e780a48cc8742209230e388f5285689d060a271d2bc40a9c5c1b23     2        25000000 lovelace + TxOutDatumNone
ef288046881fccdb0a25c608297e00e7684fe428cc888041dbfc5612455dc9b6     5        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4232 + TxOutDatumNone


Submit the transaction to withdraw the loan from the payout validator's address.

In [141]:
withdraw "$(borrower 2)" "${BORROWER_SKEY[2]}" "${BORROWER_ADDR[2]}"


Fee: Lovelace 623766
Size: 603 / 16384 = 3%
Execution units:
  Memory: 5437140 / 14000000 = 38%
  Steps: 1486014138 / 10000000000 = 14%
TxId "bdb2fa645ad5b95aeafe1b748a926a65fac11e4d8ca253ba2d14868eec2ce25c"


After withdrawing the loan, Borrower 1 has an additional 900k Djed.

In [142]:
query_borrower 2

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba615b1dc8a950f1b74ff8979e65d97f95e2b6ec1f557b4ed55e533ce2273bb6     5        2000000 lovelace + 1000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
bdb2fa645ad5b95aeafe1b748a926a65fac11e4d8ca253ba2d14868eec2ce25c     0        26376234 lovelace + TxOutDatumNone
bdb2fa645ad5b95aeafe1b748a926a65fac11e4d8ca253ba2d14868eec2ce25c     1        2430840 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4232 + 600000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone


The funds are no longer present at the payout script address.

In [143]:
query_payout "${TX_ID[$((n-6))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [144]:
query_payout "${TX_ID[$((n-4))]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


### 15. Borrower 2 pays back 720k Djed owed

#### Choose the deposit

In [145]:
BORROWER_REPAYMENT=720000

In [146]:
n=$((n+1))

In [147]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [148]:
marlowe-cli run prepare --choice-party "$(borrower 2)" \
                        --choice-name R \
                        --choice-number "$BORROWER_REPAYMENT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 1654


In [149]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: I
    choice_owner:
      role_token: I2
  - 500000
- - choice_name: L1
    choice_owner:
      role_token: A
  - 900000
- - choice_name: L2
    choice_owner:
      role_token: A
  - 600000
- - choice_name: R
    choice_owner:
      role_token: B2
  - 720000


Submit the transaction.

In [150]:
execute "${BORROWER_SKEY[2]}" "${BORROWER_ADDR[2]}"


Fee: Lovelace 1389031
Size: 5511 / 16384 = 33%
Execution units:
  Memory: 12427798 / 14000000 = 88%
  Steps: 3510318636 / 10000000000 = 35%

Transaction 29 hash: cab501c45e107faa298a3557fa3d3d4a9651b82318071b3a8d3ace817c5c4f4e


#### Make the deposit

In [151]:
n=$((n+1))

In [152]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [153]:
marlowe-cli run prepare --deposit-account "$ADMINISTRATOR_ROLE" \
                        --deposit-party "$(borrower 2)" \
                        --deposit-token "$DJED" \
                        --deposit-amount "$BORROWER_REPAYMENT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 417


In [154]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: A
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 720000
boundValues:
- - s
  - 5000000
- - a
  - 0
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 1650000


Submit the transaction.

In [155]:
execute "${BORROWER_SKEY[2]}" "${BORROWER_ADDR[2]}"


Fee: Lovelace 1105927
Size: 3076 / 16384 = 18%
Execution units:
  Memory: 10270168 / 14000000 = 73%
  Steps: 2734821753 / 10000000000 = 27%

Transaction 30 hash: 0ba4d5d24683c61c77a18f2468b027d7da7dd86c3778421a42b3ad3503f16337


The script has received the deposit.

In [156]:
query_marlowe "${TX_ID[$n]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0ba4d5d24683c61c77a18f2468b027d7da7dd86c3778421a42b3ad3503f16337     1        2000000 lovelace + 5720000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "56652eff7be8b4145706d27b41659c40d90c920b68f79a359a7a53ba46a9bb36"


Borrower has 720k Djed less the previously.

In [157]:
query_borrower 2

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0ba4d5d24683c61c77a18f2468b027d7da7dd86c3778421a42b3ad3503f16337     0        21821016 lovelace + TxOutDatumNone
0ba4d5d24683c61c77a18f2468b027d7da7dd86c3778421a42b3ad3503f16337     2        1137840 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4232 + TxOutDatumNone
0ba4d5d24683c61c77a18f2468b027d7da7dd86c3778421a42b3ad3503f16337     3        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "e50d7539222ab4b8bb940902e8b0add858d0891fc4f38be679a064856b1a04d5"
0ba4d5d24683c61c77a18f2468b027d7da7dd86c3778421a42b3ad3503f16337     4        1176630 lovelace + 280000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
cab501c45e107faa298a3557fa3d3d4a9651b82318071b3a8d3ace817c5c4f4e     3        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "415fa4

### 16. Unpause

In [158]:
unpause 4


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 475
Payment 1
  Acccount: "A"
  Payee: Account "I1"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 446836

Fee: Lovelace 1002613
Size: 1716 / 16384 = 10%
Execution units:
  Memory: 9800068 / 14000000 = 70%
  Steps: 2631340957 / 10000000000 = 26%

Transaction 31 hash: d0ab5d4b70c61a929061d5409e11562b66c70ca2092782d78a78df37accbb1aa

----- Unpause 2 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 531
Payment 1
  Acccount: "A"
  Payee: Account "I2"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 223418

Fee: Lovelace 1061090
Size: 1830 / 16384 = 11%
Execution units:
  Memory: 10503164 / 14000000 = 75%
  Steps: 2810153259 / 10000000000 = 28%

Transaction 32 hash: 03e7ab2e62b082d2880cc20d8a18b19b071b62826ab316522ea935004527047f

----- Unpause 3 -----

1664078286000 + 10800000 = 166408908600

### 17. Borrower 1 only pays back 840k of the 990k Djed owed

#### Choose the deposit

In [159]:
BORROWER_REPAYMENT=840000

In [160]:
n=$((n+1))

In [161]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [162]:
marlowe-cli run prepare --choice-party "$(borrower 1)" \
                        --choice-name R \
                        --choice-number "$BORROWER_REPAYMENT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 1847


In [163]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: I
    choice_owner:
      role_token: I2
  - 500000
- - choice_name: L1
    choice_owner:
      role_token: A
  - 900000
- - choice_name: L2
    choice_owner:
      role_token: A
  - 600000
- - choice_name: R
    choice_owner:
      role_token: B2
  - 720000
- - choice_name: R
    choice_owner:
      role_token: B1
  - 840000


Submit the transaction.

In [164]:
execute "${BORROWER_SKEY[1]}" "${BORROWER_ADDR[1]}"


Fee: Lovelace 1494184
Size: 5897 / 16384 = 35%
Execution units:
  Memory: 13591802 / 14000000 = 97%
  Steps: 3801656446 / 10000000000 = 38%

Transaction 35 hash: 5af444040f1deff349b6a56618ea39639a7c0dd8f606031ad26d1acc094709dd


#### Make the deposit

In [165]:
n=$((n+1))

In [166]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [167]:
marlowe-cli run prepare --deposit-account "$ADMINISTRATOR_ROLE" \
                        --deposit-party "$(borrower 1)" \
                        --deposit-token "$DJED" \
                        --deposit-amount "$BORROWER_REPAYMENT" \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 555


In [168]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: A
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 889746
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 446836
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 223418
boundValues:
- - s
  - 5000000
- - a
  - 720000
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 930000


Submit the transaction.

In [169]:
execute "${BORROWER_SKEY[1]}" "${BORROWER_ADDR[1]}"


Fee: Lovelace 1225474
Size: 3405 / 16384 = 20%
Execution units:
  Memory: 11655980 / 14000000 = 83%
  Steps: 3083086004 / 10000000000 = 30%

Transaction 36 hash: 496cef379f1b598302312587083ef1cc285601d6130efc2436648340d9ad4fa0


The script has received the deposit.

In [170]:
query_marlowe "${TX_ID[$n]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
496cef379f1b598302312587083ef1cc285601d6130efc2436648340d9ad4fa0     1        2000000 lovelace + 6560000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumHash ScriptDataInBabbageEra "0715c57c0b811c12c3bb7ea3eb309221cf355035506bbee4dd388b91eeafb164"


Borrower has 940k Djed less the previously.

In [171]:
query_borrower 1

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
496cef379f1b598302312587083ef1cc285601d6130efc2436648340d9ad4fa0     0        20781566 lovelace + TxOutDatumNone
496cef379f1b598302312587083ef1cc285601d6130efc2436648340d9ad4fa0     2        1137840 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4231 + TxOutDatumNone
496cef379f1b598302312587083ef1cc285601d6130efc2436648340d9ad4fa0     3        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "0132cc07f1491a207e1691990d7d19b32f13cc9c5321276dd05d7354e66c73f4"
496cef379f1b598302312587083ef1cc285601d6130efc2436648340d9ad4fa0     4        1168010 lovelace + 60000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone
5af444040f1deff349b6a56618ea39639a7c0dd8f606031ad26d1acc094709dd     3        1500000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "38b0c13

### 18. Unpause

In [172]:
unpause 4


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 555
Payment 1
  Acccount: "A"
  Payee: Account "I1"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 521309

Fee: Lovelace 1114798
Size: 1934 / 16384 = 11%
Execution units:
  Memory: 11147034 / 14000000 = 79%
  Steps: 2976314581 / 10000000000 = 29%

Transaction 37 hash: 33314ad8382c63b4847af00a947d4a778c7b1d672bb7f852d6e85788b0f38aed

----- Unpause 2 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 555
Payment 1
  Acccount: "A"
  Payee: Account "I2"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 260654

Fee: Lovelace 1131168
Size: 1934 / 16384 = 11%
Execution units:
  Memory: 11358000 / 14000000 = 81%
  Steps: 3034540660 / 10000000000 = 30%

Transaction 38 hash: 238aae955ef884602945a0bcb0f1116f0ac2a770b09a21a768a9e2003035357e

----- Unpause 3 -----

1664078286000 + 10800000 = 16640890860

### 19. Administrator closes the pool

In [173]:
n=$((n+1))

In [174]:
set_validity_interval $((n-1))

1664078286000 + 10800000 = 1664089086000


In [175]:
marlowe-cli run prepare --choice-party "$ADMINISTRATOR_ROLE" \
                        --choice-name T \
                        --choice-number 1 \
                        --invalid-before "$INVALID_BEFORE" \
                        --invalid-hereafter "$INVALID_HEREAFTER" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe" \
                        --out-file     "$BASENAME-$n.marlowe" \
                        --print-stats


Datum size: 568


In [176]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: A
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 107783
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 968145
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 484072
boundValues:
- - s
  - 5000000
- - a
  - 1560000
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 90000


Submit the transaction.

In [177]:
execute "$ADMINISTRATOR_SKEY" "$ADMINISTRATOR_ADDR"


Fee: Lovelace 1085471
Size: 3225 / 16384 = 19%
Execution units:
  Memory: 9986480 / 14000000 = 71%
  Steps: 2648838341 / 10000000000 = 26%

Transaction 41 hash: 70b3e2ec47b99d6cdaa0452457249088e6aac243d2adb814ae8c79d9ead3b35c


### 20. Unpause

In [178]:
unpause 8


----- Unpause 1 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 568

Fee: Lovelace 786428
Size: 1879 / 16384 = 11%
Execution units:
  Memory: 6895462 / 14000000 = 49%
  Steps: 1857956160 / 10000000000 = 18%

Transaction 42 hash: 5d4202110d5f9084a2266d6e4859f3c35b6a6fe0921721c5df5719b315317511

----- Unpause 2 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 568
Payment 1
  Acccount: "S1"
  Payee: Account "A"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 90000

Fee: Lovelace 979888
Size: 1937 / 16384 = 11%
Execution units:
  Memory: 9397020 / 14000000 = 67%
  Steps: 2503827365 / 10000000000 = 25%

Transaction 43 hash: 4cc9f9a857134c5c1b583378b8458d926897e5be554924fc3dee476df398732b

----- Unpause 3 -----

1664078286000 + 10800000 = 1664089086000

Datum size: 568
Payment 1
  Acccount: "A"
  Payee: Account "I1"
  Ada: Lovelace {getLovelace = 0}
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."Dj

The contract has closed.

In [179]:
show_contract $n

close
...


There are no funds at the Marlowe script address.

In [180]:
query_marlowe "{$TX_ID[$n]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


### 21. Everyone withdraws their funds

In [181]:
echo "----- Administrator -----"
withdraw "$ADMINISTRATOR_ROLE" "$ADMINISTRATOR_SKEY" "$ADMINISTRATOR_ADDR"
echo
marlowe-cli util clean --required-signer "$ADMINISTRATOR_SKEY" --change-address "$ADMINISTRATOR_ADDR" --out-file /dev/null --submit 600
echo
query_administrator

----- Administrator -----

Fee: Lovelace 373647
Size: 546 / 16384 = 3%
Execution units:
  Memory: 2234236 / 14000000 = 15%
  Steps: 614953119 / 10000000000 = 6%
TxId "ea6d36d5ecc98366dd1eefd053a49ca8cf26c9ab8c2cfce88168d63bc4e769c9"

TxId "f858aac63a97aeb8f49f381f4b5c0bad704dbf3cb0f9986a2ba6bb8446de4761"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f858aac63a97aeb8f49f381f4b5c0bad704dbf3cb0f9986a2ba6bb8446de4761     0        429323052 lovelace + TxOutDatumNone
f858aac63a97aeb8f49f381f4b5c0bad704dbf3cb0f9986a2ba6bb8446de4761     1        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.41 + TxOutDatumNone
f858aac63a97aeb8f49f381f4b5c0bad704dbf3cb0f9986a2ba6bb8446de4761     2        2000000 lovelace + 51754 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone


In [182]:
for j in `seq 1 $N_STABILIZERS`
do
  echo
  echo "----- Stabilizer $j -----"
  withdraw "$(stabilizer $j)" "${STABILIZER_SKEY[$j]}" "${STABILIZER_ADDR[$j]}"
  echo
  marlowe-cli util clean --required-signer "${STABILIZER_SKEY[$j]}" --change-address "${STABILIZER_ADDR[$j]}" --out-file /dev/null --submit 600
  echo
  query_stabilizer $j
  echo
done


----- Stabilizer 1 -----

Fee: Lovelace 373311
Size: 550 / 16384 = 3%
Execution units:
  Memory: 2227634 / 14000000 = 15%
  Steps: 613137542 / 10000000000 = 6%
TxId "953606c0d6e9a80b254b8da69476615f2c4173aae48b25c1bee91fad8db455ae"

TxId "7fc7e19ffcde6147ac080516c34dd9ac880905ae8d1994e5f1597de08699d727"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7fc7e19ffcde6147ac080516c34dd9ac880905ae8d1994e5f1597de08699d727     0        23477244 lovelace + TxOutDatumNone
7fc7e19ffcde6147ac080516c34dd9ac880905ae8d1994e5f1597de08699d727     1        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.5331 + TxOutDatumNone
7fc7e19ffcde6147ac080516c34dd9ac880905ae8d1994e5f1597de08699d727     2        2000000 lovelace + 9910000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone



In [183]:
for j in `seq 1 $N_INVESTORS`
do
  echo
  echo "----- Investor $j -----"
  withdraw "$(investor $j)" "${INVESTOR_SKEY[$j]}" "${INVESTOR_ADDR[$j]}"
  echo
  marlowe-cli util clean --required-signer "${INVESTOR_SKEY[$j]}" --change-address "${INVESTOR_ADDR[$j]}" --out-file /dev/null --submit 600
  echo
  query_investor $j
  echo
done


----- Investor 1 -----

Fee: Lovelace 373311
Size: 550 / 16384 = 3%
Execution units:
  Memory: 2227634 / 14000000 = 15%
  Steps: 613137542 / 10000000000 = 6%
TxId "a320e488251eb6055c78c5dd38311dcdf5e4e8d5c864bf780110e6112008a936"

TxId "92327895e115f145d99b94c20f91191001621cb9298b21e5d1dcb4923860f3fe"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
92327895e115f145d99b94c20f91191001621cb9298b21e5d1dcb4923860f3fe     0        23289303 lovelace + TxOutDatumNone
92327895e115f145d99b94c20f91191001621cb9298b21e5d1dcb4923860f3fe     1        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4931 + TxOutDatumNone
92327895e115f145d99b94c20f91191001621cb9298b21e5d1dcb4923860f3fe     2        2000000 lovelace + 5090890 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone


----- Investor 2 -----

Fee: Lovelac

In [184]:
for j in `seq 1 $N_BORROWERS`
do
  echo
  echo "----- Borrower $j -----"
  echo
  marlowe-cli util clean --required-signer "${BORROWER_SKEY[$j]}" --change-address "${BORROWER_ADDR[$j]}" --out-file /dev/null --submit 600
  echo
  query_borrower $j
  echo
done


----- Borrower 1 -----

TxId "27d56de351ea061121c40539a10d760f4089e66257d60f66996d0e705b2e58f4"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
27d56de351ea061121c40539a10d760f4089e66257d60f66996d0e705b2e58f4     0        23879543 lovelace + TxOutDatumNone
27d56de351ea061121c40539a10d760f4089e66257d60f66996d0e705b2e58f4     1        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4231 + TxOutDatumNone
27d56de351ea061121c40539a10d760f4089e66257d60f66996d0e705b2e58f4     2        2000000 lovelace + 1060000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone


----- Borrower 2 -----

TxId "2d9c0ffe3adb0810a5b5266090a2f15420078321476dbe46eb2ea2cb644951a4"

                           TxHash                                 TxIx        Amount
-------------------------------------------------------------